<a href="https://colab.research.google.com/github/sunshineluyao/ic-gov-visualization/blob/main/code/IC_Gov_Visualization_GoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I Set up

In [5]:
pip install -U kaleido #tools for explorting figures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install plotapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install igviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
pip install pyecharts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import collections
import json
import os
import pickle
import string
import warnings
from multiprocessing.pool import ThreadPool
from operator import itemgetter

import igviz as ig
import networkx as nx
import nltk # https://realpython.com/nltk-nlp-python/
import pandas as pd
import plotapi
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pyecharts.options as opts
import regex
import requests
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from plotapi import SplitChord, Terminus
from plotly.subplots import make_subplots
from pyecharts.charts import WordCloud
from tqdm import tqdm

warnings.filterwarnings('ignore')
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
#@title Get your Plot API Key before moving forward
PlotAPI = "b0368160-7e02-4566-9dec-1829bd5b070b" #@param {type:"string"}


! Note: Our Code to Generate Multi-media Assets Require a PlotAPI Business License

In [84]:
import plotapi 
plotapi.api_key("b0368160-7e02-4566-9dec-1829bd5b070b")

Your PlotAPI API key has been saved in your local environment. You will not need to set it again.


In [12]:
# see all the figure templates: https://plotly.com/python/templates/
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [13]:
pio.templates.default = 'plotly_dark'

In [14]:
# mount google drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# print the current directory
!pwd

/content


In [16]:
#@title path
#create a project folder and identify the path
path = "/content/drive/MyDrive/ic-gov-visualization" #@param {type:"string"}


In [17]:
#go to your project folder 
%cd /content/drive/MyDrive/ic-gov-visualization

/content/drive/MyDrive/ic-gov-visualization


In [18]:
# print the current directory again to make sure that you are in the project folder
!pwd  

/content/drive/MyDrive/ic-gov-visualization


In [19]:
import pandas as pd
import numpy as np

# Part II: Query and Process Data

In [89]:
base_url = 'https://ic-api.internetcomputer.org'

fetched: proposal 5908
fetched: proposal 78950
fetched: proposal 5922
fetched: proposal 5898
fetched: proposal 5915
fetched: proposal 5923fetched: proposal 5894
fetched: proposal 78987
fetched: proposal 78971

fetched: proposal 79011
fetched: proposal 5918
fetched: proposal 79020
fetched: proposal 5904
fetched: proposal 78982fetched: proposal 5901

fetched: proposal 79010
fetched: proposal 78978
fetched: proposal 78998
fetched: proposal 79002
fetched: proposal 78991
fetched: proposal 78986
fetched: proposal 79027
fetched: proposal 79004fetched: proposal 5895
fetched: proposal 78997fetched: proposal 5897
fetched: proposal 79001

fetched: proposal 5877
fetched: proposal 79006
fetched: proposal 79009
fetched: proposal 79013
fetched: proposal 79014
fetched: proposal 5889
fetched: proposal 5903
fetched: proposal 79029
fetched: proposal 5928fetched: proposal 79024
fetched: proposal 79017
fetched: proposal 79026


fetched: proposal 5914fetched: proposal 79021
fetched: proposal 79033
fetched: 

In [93]:
# get total proposal counts
res = requests.get(f'{base_url}/api/nns/proposals-count')
proposals_count = json.loads(res.text)['proposals_count']

fetched: proposal 80083
fetched: proposal 6727
fetched: proposal 80097
fetched: proposal 80096
fetched: proposal 6748
fetched: proposal 6746fetched: proposal 6753

fetched: proposal 6729
fetched: proposal 80102
fetched: proposal 80099
fetched: proposal 80107
fetched: proposal 6734
fetched: proposal 80123
fetched: proposal 80126fetched: proposal 80100
fetched: proposal 80128
fetched: proposal 80108
fetched: proposal 6771

fetched: proposal 80117
fetched: proposal 6774
fetched: proposal 80104
fetched: proposal 80136
fetched: proposal 6732
fetched: proposal 80110
fetched: proposal 6769
fetched: proposal 6738
fetched: proposal 6780
fetched: proposal 80119
fetched: proposal 80098
fetched: proposal 80106
fetched: proposal 80113
fetched: proposal 80115
fetched: proposal 6775fetched: proposal 80131
fetched: proposal 80129
fetched: proposal 80118
fetched: proposal 80105
fetched: proposal 6742fetched: proposal 80145
fetched: proposal 6770
fetched: proposal 80137

fetched: proposal 6777
fetched: 

In [94]:
print(f'Total Proposals: {proposals_count}')

fetched: proposal 6802
fetched: proposal 80200fetched: proposal 80227
fetched: proposal 6806
Total Proposals: 99001
fetched: proposal 80203

fetched: proposal 80224
fetched: proposal 6804
fetched: proposal 80204
fetched: proposal 80237
fetched: proposal 80208

fetched: proposal 80222
fetched: proposal 6844
fetched: proposal 6839fetched: proposal 80209
fetched: proposal 80202
fetched: proposal 6842

fetched: proposal 6813
fetched: proposal 80205
fetched: proposal 80213
fetched: proposal 80207
fetched: proposal 80220
fetched: proposal 6841fetched: proposal 80236


In [95]:
# fetch proposals
def get_proposal(id):
    url = base_url + '/api/v3/proposals/' + str(id)
    res = requests.get(url)
    res_dict = json.loads(res.text)
    return res_dict


def get_neuron(id):
    url = f'https://ic-api.internetcomputer.org/api/v3/neurons/{id}'
    res = requests.get(url)
    res_dict = json.loads(res.text)
    return res_dict

fetched: proposal 7092
fetched: proposal 80596
fetched: proposal 80599
fetched: proposal 80597
fetched: proposal 80598
fetched: proposal 80594
fetched: proposal 7098
fetched: proposal 80593
fetched: proposal 7108
fetched: proposal 80606
fetched: proposal 80607
fetched: proposal 7109
fetched: proposal 80600
fetched: proposal 7111
fetched: proposal 80601
fetched: proposal 80602
fetched: proposal 80609
fetched: proposal 7110
fetched: proposal 80603
fetched: proposal 80615
fetched: proposal 80605
fetched: proposal 80612
fetched: proposal 80604
fetched: proposal 80610
fetched: proposal 80616
fetched: proposal 80608
fetched: proposal 80634
fetched: proposal 80622
fetched: proposal 80617
fetched: proposal 80635
fetched: proposal 7121
fetched: proposal 80620
fetched: proposal 80613
fetched: proposal 80619


In [97]:
proposals = []
proposal_ids = [i for i in range(proposals_count)]


def dummy(id):
    res = get_proposal(id)
    proposals.append(dict(
        id=id,
        data=res
    ))
    print(f'fetched: proposal {id}')


# enable multithreading for faster download
pool = ThreadPool(50)

pool.imap_unordered(dummy, proposal_ids)


fetched: proposal 81647fetched: proposal 81645fetched: proposal 7875fetched: proposal 81661fetched: proposal 7837fetched: proposal 81642

fetched: proposal 7866

fetched: proposal 7883
fetched: proposal 81652

fetched: proposal 81654
fetched: proposal 81656
fetched: proposal 81667


fetched: proposal 81672

fetched: proposal 81680
fetched: proposal 81675
fetched: proposal 81669
fetched: proposal 7873
fetched: proposal 81658
fetched: proposal 7880fetched: proposal 81648
fetched: proposal 81668
fetched: proposal 7901
fetched: proposal 81682
fetched: proposal 7913

fetched: proposal 7889

fetched: proposal 81659
fetched: proposal 81653
fetched: proposal 7871
fetched: proposal 81673
fetched: proposal 7909
fetched: proposal 81646
fetched: proposal 7874
fetched: proposal 81660
fetched: proposal 7899
fetched: proposal 7894
fetched: proposal 7891
fetched: proposal 81666
fetched: proposal 7878
fetched: proposal 81629
fetched: proposal 81664fetched: proposal 81691
fetched: proposal 81674fetched:

fetched: proposal 81993
fetched: proposal 8192fetched: proposal 82094
fetched: proposal 8193
fetched: proposal 82077fetched: proposal 8169
fetched: proposal 8196
fetched: proposal 8155
fetched: proposal 82046
fetched: proposal 82079
fetched: proposal 82083
fetched: proposal 82062
fetched: proposal 82087

fetched: proposal 8147

fetched: proposal 82090
fetched: proposal 8184

fetched: proposal 8218
fetched: proposal 8214
fetched: proposal 82097
fetched: proposal 8201
fetched: proposal 82098
fetched: proposal 82093
fetched: proposal 82110
fetched: proposal 82096
fetched: proposal 82091


In [98]:
# create an empty pickle file
import os
filename = "data/nns_data_raw.pkl"
os.makedirs(os.path.dirname(filename), exist_ok=True)

fetched: proposal 8360
fetched: proposal 82273
fetched: proposal 8354
fetched: proposal 161
fetched: proposal 156
fetched: proposal 82281
fetched: proposal 162
fetched: proposal 8358
fetched: proposal 167
fetched: proposal 82282
fetched: proposal 163fetched: proposal 164
fetched: proposal 168
fetched: proposal 172
fetched: proposal 165

fetched: proposal 170
fetched: proposal 166
fetched: proposal 173
fetched: proposal 169
fetched: proposal 174
fetched: proposal 82283
fetched: proposal 8361
fetched: proposal 8359
fetched: proposal 82285
fetched: proposal 171
fetched: proposal 82284fetched: proposal 177

fetched: proposal 176
fetched: proposal 82286
fetched: proposal 82289
fetched: proposal 178
fetched: proposal 175
fetched: proposal 185


In [99]:
# dump the data to the pickle file
pickle.dump(proposals, open('data/nns_data_raw.pkl', 'wb'))

fetched: proposal 82400
fetched: proposal 8448
fetched: proposal 82405
fetched: proposal 82397
fetched: proposal 82399
fetched: proposal 8454
fetched: proposal 8451
fetched: proposal 82409
fetched: proposal 8465
fetched: proposal 82407
fetched: proposal 302fetched: proposal 82408
fetched: proposal 8455

fetched: proposal 303
fetched: proposal 82403
fetched: proposal 82410
fetched: proposal 82402
fetched: proposal 304fetched: proposal 82404
fetched: proposal 82406
fetched: proposal 8452

fetched: proposal 8456
fetched: proposal 305
fetched: proposal 8457
fetched: proposal 8466
fetched: proposal 82411
fetched: proposal 82415
fetched: proposal 82412
fetched: proposal 82416
fetched: proposal 82413
fetched: proposal 8467
fetched: proposal 8462
fetched: proposal 82414
fetched: proposal 8458
fetched: proposal 8460fetched: proposal 8470
fetched: proposal 270

fetched: proposal 306
fetched: proposal 307
fetched: proposal 310
fetched: proposal 309
fetched: proposal 82419
fetched: proposal 82422


In [100]:
# sort data by proposal id
data = pickle.load(open('data/nns_data_raw.pkl', 'rb'))
data.sort(key=itemgetter('id'))
json.dump(data, open('data/proposals.json', 'w'))

fetched: proposal 82476
fetched: proposal 386
fetched: proposal 82477
fetched: proposal 8521
fetched: proposal 82479
fetched: proposal 383

fetched: proposal 385fetched: proposal 384

fetched: proposal 8526
fetched: proposal 82474
fetched: proposal 82480
fetched: proposal 82481
fetched: proposal 378
fetched: proposal 8530fetched: proposal 8515

fetched: proposal 382
fetched: proposal 388
fetched: proposal 8522
fetched: proposal 387
fetched: proposal 82484
fetched: proposal 389
fetched: proposal 82482
fetched: proposal 82486
fetched: proposal 82485
fetched: proposal 82489
fetched: proposal 390
fetched: proposal 82490
fetched: proposal 392
fetched: proposal 391
fetched: proposal 393
fetched: proposal 82487
fetched: proposal 394
fetched: proposal 8533
fetched: proposal 396
fetched: proposal 395
fetched: proposal 398fetched: proposal 82491

fetched: proposal 8555
fetched: proposal 401
fetched: proposal 397
fetched: proposal 399
fetched: proposal 400
fetched: proposal 402
fetched: proposal 

In [101]:
# filter empty data and save the data as json files
proposals_no_empty = []

for item in data:
    if (item['data'].get('code') != 404):
        proposals_no_empty.append(
            item['data']
        )
json.dump(proposals_no_empty, open('data/proposals_no_empty.json', 'w'))


fetched: proposal 82614
fetched: proposal 8631
fetched: proposal 513
fetched: proposal 82613
fetched: proposal 516
fetched: proposal 517
fetched: proposal 514
fetched: proposal 82617
fetched: proposal 521
fetched: proposal 523
fetched: proposal 519
fetched: proposal 8637
fetched: proposal 525
fetched: proposal 82615
fetched: proposal 518
fetched: proposal 520
fetched: proposal 82616
fetched: proposal 515
fetched: proposal 526
fetched: proposal 522
fetched: proposal 529
fetched: proposal 8641
fetched: proposal 532
fetched: proposal 528
fetched: proposal 82618
fetched: proposal 82622
fetched: proposal 8640
fetched: proposal 534
fetched: proposal 82621
fetched: proposal 8644
fetched: proposal 524
fetched: proposal 82619
fetched: proposal 536
fetched: proposal 527
fetched: proposal 8635fetched: proposal 535
fetched: proposal 537
fetched: proposal 530

fetched: proposal 533
fetched: proposal 543
fetched: proposal 82623
fetched: proposal 531
fetched: proposal 8649
fetched: proposal 540
fetch

In [102]:
# read the json file data as pandas dataframe
df = pd.read_json('data/proposals_no_empty.json')
df

fetched: proposal 82859
fetched: proposal 778
fetched: proposal 777
fetched: proposal 781
fetched: proposal 82856fetched: proposal 776
fetched: proposal 774

fetched: proposal 82858
fetched: proposal 780
fetched: proposal 784
fetched: proposal 82857
fetched: proposal 8825
fetched: proposal 8831
fetched: proposal 783
fetched: proposal 82860
fetched: proposal 787
fetched: proposal 82864
fetched: proposal 82854
fetched: proposal 786
fetched: proposal 779fetched: proposal 785

fetched: proposal 82863
fetched: proposal 8826
fetched: proposal 789
fetched: proposal 782
fetched: proposal 8828
fetched: proposal 82862
fetched: proposal 8832
fetched: proposal 8824
fetched: proposal 82861
fetched: proposal 8827
fetched: proposal 8829
fetched: proposal 8837
fetched: proposal 8836fetched: proposal 8835fetched: proposal 790

fetched: proposal 788
fetched: proposal 82865

fetched: proposal 8834fetched: proposal 82866

fetched: proposal 794
fetched: proposal 82867
fetched: proposal 793
fetched: proposa

,action,action_nns_function,deadline_timestamp_seconds,decided_timestamp_seconds,executed_timestamp_seconds,failed_timestamp_seconds,id,known_neurons_ballots,latest_tally,payload,...,proposer,reject_cost_e8s,reward_status,settled_at,status,summary,title,topic,updated_at,url
0,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340878,1620340878,0.0,1,[],"{'no': 0, 'timestamp_seconds': 1620340878, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade ledger canister to git commit 8a560f95...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.155180,https://github.com/dfinity/nns-proposals/blob/...
1,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340965,1620340965,0.0,2,[],"{'no': 0, 'timestamp_seconds': 1620340965, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade registry canister to git commit 8a560f...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.157946,https://github.com/dfinity/nns-proposals/blob/...
2,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341025,1620341025,0.0,3,[],"{'no': 0, 'timestamp_seconds': 1620341025, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade governance canister to git commit 8a56...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.158920,https://github.com/dfinity/nns-proposals/blob/...
3,ExecuteNnsFunction,NnsRootUpgrade,NaN,1620341096,1620341110,0.0,4,[],"{'no': 0, 'timestamp_seconds': 1620341096, 'to...",{},...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade root canister to git commit 8a560f9510...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.159834,https://github.com/dfinity/nns-proposals/blob/...
4,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341144,1620341144,0.0,5,[],"{'no': 0, 'timestamp_seconds': 1620341144, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade cycles-minting canister to git commit ...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.160766,https://github.com/dfinity/nns-proposals/blob/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,ExecuteNnsFunction,IcpXdrConversionRate,1.664003e+09,1663960248,1663960248,0.0,68169,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663960248, 'to...","{'data_source': {'icp': ['Binance'], 'sdr': 'x...",...,26.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7801.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.550190,
1013,ExecuteNnsFunction,IcpXdrConversionRate,1.664004e+09,1663960883,1663960883,0.0,68170,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663960883, 'to...","{'data_source': {'icp': ['Huobi'], 'sdr': 'xe....",...,25.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7723.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.530476,
1014,ExecuteNnsFunction,IcpXdrConversionRate,1.664005e+09,1663961552,1663961552,0.0,68171,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663961552, 'to...","{'data_source': {'icp': ['Binance'], 'sdr': 'x...",...,24.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7646.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.510036,
1015,ExecuteNnsFunction,IcpXdrConversionRate,1.664005e+09,1663962087,1663962087,0.0,68172,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663962087, 'to...","{'data_source': {'icp': ['Huobi'], 'sdr': 'xe....",...,22.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7646.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.490062,


fetched: proposal 8877

fetched: proposal 82918
fetched: proposal 847

fetched: proposal 848
fetched: proposal 850
fetched: proposal 852fetched: proposal 82916
fetched: proposal 8891

fetched: proposal 82919fetched: proposal 856
fetched: proposal 8880
fetched: proposal 82921


In [103]:
df.to_csv("proposals_no_empty.csv")
df.to_pickle("proposals_no_empty.pkl")

fetched: proposal 83690
fetched: proposal 83688
fetched: proposal 83686
fetched: proposal 1671
fetched: proposal 1679
fetched: proposal 1676
fetched: proposal 83691
fetched: proposal 9516
fetched: proposal 1673
fetched: proposal 1678fetched: proposal 9522
fetched: proposal 83727
fetched: proposal 1680

fetched: proposal 83693
fetched: proposal 1675
fetched: proposal 9518
fetched: proposal 1677
fetched: proposal 83695
fetched: proposal 1681
fetched: proposal 9517
fetched: proposal 1682
fetched: proposal 83697
fetched: proposal 83694fetched: proposal 83696

fetched: proposal 9524
fetched: proposal 83699
fetched: proposal 1683
fetched: proposal 1684
fetched: proposal 83698
fetched: proposal 1687
fetched: proposal 83700
fetched: proposal 1685
fetched: proposal 1686
fetched: proposal 1691
fetched: proposal 1690
fetched: proposal 1689
fetched: proposal 1688
fetched: proposal 83702
fetched: proposal 1692
fetched: proposal 83701
fetched: proposal 9535
fetched: proposal 83703
fetched: proposal 

In [104]:
df = pd.read_pickle("proposals_no_empty.pkl")
df

fetched: proposal 9576
fetched: proposal 1759
fetched: proposal 83767fetched: proposal 1763

fetched: proposal 83770
fetched: proposal 1765
fetched: proposal 83771
fetched: proposal 1803
fetched: proposal 83772
fetched: proposal 1764fetched: proposal 9579

fetched: proposal 1761
fetched: proposal 83764
fetched: proposal 1766
fetched: proposal 83775
fetched: proposal 1768
fetched: proposal 1767
fetched: proposal 83774
fetched: proposal 83773
fetched: proposal 83780
fetched: proposal 9583
fetched: proposal 9596
fetched: proposal 83779fetched: proposal 83777
fetched: proposal 9592

fetched: proposal 1773
fetched: proposal 83782
fetched: proposal 83778
fetched: proposal 83776
fetched: proposal 1771fetched: proposal 1778
fetched: proposal 1770

fetched: proposal 1769
fetched: proposal 1777
fetched: proposal 83784
fetched: proposal 1775
fetched: proposal 1780
fetched: proposal 1772
fetched: proposal 9585
fetched: proposal 83788
fetched: proposal 83785
fetched: proposal 1776
fetched: proposal

,action,action_nns_function,deadline_timestamp_seconds,decided_timestamp_seconds,executed_timestamp_seconds,failed_timestamp_seconds,id,known_neurons_ballots,latest_tally,payload,...,proposer,reject_cost_e8s,reward_status,settled_at,status,summary,title,topic,updated_at,url
0,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340878,1620340878,0.0,1,[],"{'no': 0, 'timestamp_seconds': 1620340878, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade ledger canister to git commit 8a560f95...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.155180,https://github.com/dfinity/nns-proposals/blob/...
1,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340965,1620340965,0.0,2,[],"{'no': 0, 'timestamp_seconds': 1620340965, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade registry canister to git commit 8a560f...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.157946,https://github.com/dfinity/nns-proposals/blob/...
2,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341025,1620341025,0.0,3,[],"{'no': 0, 'timestamp_seconds': 1620341025, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade governance canister to git commit 8a56...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.158920,https://github.com/dfinity/nns-proposals/blob/...
3,ExecuteNnsFunction,NnsRootUpgrade,NaN,1620341096,1620341110,0.0,4,[],"{'no': 0, 'timestamp_seconds': 1620341096, 'to...",{},...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade root canister to git commit 8a560f9510...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.159834,https://github.com/dfinity/nns-proposals/blob/...
4,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341144,1620341144,0.0,5,[],"{'no': 0, 'timestamp_seconds': 1620341144, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade cycles-minting canister to git commit ...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.160766,https://github.com/dfinity/nns-proposals/blob/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,ExecuteNnsFunction,IcpXdrConversionRate,1.664003e+09,1663960248,1663960248,0.0,68169,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663960248, 'to...","{'data_source': {'icp': ['Binance'], 'sdr': 'x...",...,26.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7801.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.550190,
1013,ExecuteNnsFunction,IcpXdrConversionRate,1.664004e+09,1663960883,1663960883,0.0,68170,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663960883, 'to...","{'data_source': {'icp': ['Huobi'], 'sdr': 'xe....",...,25.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7723.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.530476,
1014,ExecuteNnsFunction,IcpXdrConversionRate,1.664005e+09,1663961552,1663961552,0.0,68171,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663961552, 'to...","{'data_source': {'icp': ['Binance'], 'sdr': 'x...",...,24.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7646.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.510036,
1015,ExecuteNnsFunction,IcpXdrConversionRate,1.664005e+09,1663962087,1663962087,0.0,68172,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663962087, 'to...","{'data_source': {'icp': ['Huobi'], 'sdr': 'xe....",...,22.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7646.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.490062,


fetched: proposal 83819
fetched: proposal 1811
fetched: proposal 83815
fetched: proposal 1812
fetched: proposal 83818
fetched: proposal 9631
fetched: proposal 83813
fetched: proposal 83822
fetched: proposal 1816
fetched: proposal 83821
fetched: proposal 83826
fetched: proposal 1817
fetched: proposal 9636
fetched: proposal 83827
fetched: proposal 1815
fetched: proposal 83830

In [105]:
# display interaction tables: https://colab.research.google.com/notebooks/data_table.ipynb#scrollTo=IdBulJJa8Jta
from google.colab import data_table
data_table.DataTable(df, include_index=False, num_rows_per_page=10)

fetched: proposal 83981
fetched: proposal 83978
fetched: proposal 83976
fetched: proposal 1985
fetched: proposal 9752fetched: proposal 1983

fetched: proposal 1986
fetched: proposal 83982
fetched: proposal 1987
fetched: proposal 83983
fetched: proposal 83985
fetched: proposal 9753
fetched: proposal 9766
fetched: proposal 1988
fetched: proposal 1990
fetched: proposal 1995
fetched: proposal 83984
fetched: proposal 1991
fetched: proposal 1993
fetched: proposal 83986fetched: proposal 1989

fetched: proposal 1994
fetched: proposal 1996
fetched: proposal 83989
fetched: proposal 83991
fetched: proposal 83987fetched: proposal 1992
fetched: proposal 2002
fetched: proposal 83988
fetched: proposal 1997
fetched: proposal 83992

fetched: proposal 2008
fetched: proposal 9756fetched: proposal 2006
fetched: proposal 2001

fetched: proposal 1999
fetched: proposal 9790
fetched: proposal 2007
fetched: proposal 84002
fetched: proposal 83996
fetched: proposal 9786
fetched: proposal 1998
fetched: proposal 2

,action,action_nns_function,deadline_timestamp_seconds,decided_timestamp_seconds,executed_timestamp_seconds,failed_timestamp_seconds,id,known_neurons_ballots,latest_tally,payload,...,proposer,reject_cost_e8s,reward_status,settled_at,status,summary,title,topic,updated_at,url
0,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340878,1620340878,0.0,1,[],"{'no': 0, 'timestamp_seconds': 1620340878, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade ledger canister to git commit 8a560f95...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.155180,https://github.com/dfinity/nns-proposals/blob/...
1,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340965,1620340965,0.0,2,[],"{'no': 0, 'timestamp_seconds': 1620340965, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade registry canister to git commit 8a560f...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.157946,https://github.com/dfinity/nns-proposals/blob/...
2,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341025,1620341025,0.0,3,[],"{'no': 0, 'timestamp_seconds': 1620341025, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade governance canister to git commit 8a56...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.158920,https://github.com/dfinity/nns-proposals/blob/...
3,ExecuteNnsFunction,NnsRootUpgrade,NaN,1620341096,1620341110,0.0,4,[],"{'no': 0, 'timestamp_seconds': 1620341096, 'to...",{},...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade root canister to git commit 8a560f9510...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.159834,https://github.com/dfinity/nns-proposals/blob/...
4,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341144,1620341144,0.0,5,[],"{'no': 0, 'timestamp_seconds': 1620341144, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade cycles-minting canister to git commit ...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.160766,https://github.com/dfinity/nns-proposals/blob/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,ExecuteNnsFunction,IcpXdrConversionRate,1.664003e+09,1663960248,1663960248,0.0,68169,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663960248, 'to...","{'data_source': {'icp': ['Binance'], 'sdr': 'x...",...,26.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7801.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.550190,
1013,ExecuteNnsFunction,IcpXdrConversionRate,1.664004e+09,1663960883,1663960883,0.0,68170,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663960883, 'to...","{'data_source': {'icp': ['Huobi'], 'sdr': 'xe....",...,25.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7723.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.530476,
1014,ExecuteNnsFunction,IcpXdrConversionRate,1.664005e+09,1663961552,1663961552,0.0,68171,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663961552, 'to...","{'data_source': {'icp': ['Binance'], 'sdr': 'x...",...,24.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7646.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.510036,
1015,ExecuteNnsFunction,IcpXdrConversionRate,1.664005e+09,1663962087,1663962087,0.0,68172,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663962087, 'to...","{'data_source': {'icp': ['Huobi'], 'sdr': 'xe....",...,22.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7646.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.490062,


fetched: proposal 9863
fetched: proposal 84103
fetched: proposal 84100
fetched: proposal 9862
fetched: proposal 9854
fetched: proposal 84105
fetched: proposal 2113fetched: proposal 9859

fetched: proposal 84102
fetched: proposal 84107
fetched: proposal 9865
fetched: proposal 2120
fetched: proposal 9856
fetched: proposal 84108
fetched: proposal 84110
fetched: proposal 2121fetched: proposal 84106
fetched: proposal 84070
fetched: proposal 2125

fetched: proposal 84112
fetched: proposal 2129


# Part III Visualizations

## 3.1. Wordcloud - Proposal Summary

In [20]:
df = pd.read_pickle("proposals_no_empty.pkl")
df

,action,action_nns_function,deadline_timestamp_seconds,decided_timestamp_seconds,executed_timestamp_seconds,failed_timestamp_seconds,id,known_neurons_ballots,latest_tally,payload,...,proposer,reject_cost_e8s,reward_status,settled_at,status,summary,title,topic,updated_at,url
0,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340878,1620340878,0.0,1,[],"{'no': 0, 'timestamp_seconds': 1620340878, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade ledger canister to git commit 8a560f95...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.155180,https://github.com/dfinity/nns-proposals/blob/...
1,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340965,1620340965,0.0,2,[],"{'no': 0, 'timestamp_seconds': 1620340965, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade registry canister to git commit 8a560f...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.157946,https://github.com/dfinity/nns-proposals/blob/...
2,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341025,1620341025,0.0,3,[],"{'no': 0, 'timestamp_seconds': 1620341025, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade governance canister to git commit 8a56...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.158920,https://github.com/dfinity/nns-proposals/blob/...
3,ExecuteNnsFunction,NnsRootUpgrade,NaN,1620341096,1620341110,0.0,4,[],"{'no': 0, 'timestamp_seconds': 1620341096, 'to...",{},...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade root canister to git commit 8a560f9510...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.159834,https://github.com/dfinity/nns-proposals/blob/...
4,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341144,1620341144,0.0,5,[],"{'no': 0, 'timestamp_seconds': 1620341144, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade cycles-minting canister to git commit ...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.160766,https://github.com/dfinity/nns-proposals/blob/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,ExecuteNnsFunction,IcpXdrConversionRate,1.664003e+09,1663960248,1663960248,0.0,68169,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663960248, 'to...","{'data_source': {'icp': ['Binance'], 'sdr': 'x...",...,26.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7801.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.550190,
1013,ExecuteNnsFunction,IcpXdrConversionRate,1.664004e+09,1663960883,1663960883,0.0,68170,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663960883, 'to...","{'data_source': {'icp': ['Huobi'], 'sdr': 'xe....",...,25.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7723.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.530476,
1014,ExecuteNnsFunction,IcpXdrConversionRate,1.664005e+09,1663961552,1663961552,0.0,68171,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663961552, 'to...","{'data_source': {'icp': ['Binance'], 'sdr': 'x...",...,24.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7646.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.510036,
1015,ExecuteNnsFunction,IcpXdrConversionRate,1.664005e+09,1663962087,1663962087,0.0,68172,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663962087, 'to...","{'data_source': {'icp': ['Huobi'], 'sdr': 'xe....",...,22.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7646.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.490062,


In [21]:
with open('data/proposal_summary_text.txt', 'w') as f:
    f.write(' '.join(list(df['summary'].values)))

In [22]:
# cleaning data for wordcloud
df_wordcloud = df[['topic', 'summary']]
df_wordcloud = df_wordcloud.groupby('topic').aggregate(' '.join).reset_index()
df_wordcloud.to_csv('data/topic_summary_joined.csv')

In [23]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
# remove non-utf8 characters
def remove_non_utf8(x: str):
    x = x.encode('utf-8', errors='ignore').decode('utf-8')
    x = regex.sub(r'[^\x00-\x7f]', u'', x)
    x = regex.sub(r"http\S+", "", x)
    x = x.translate(str.maketrans('', '', string.punctuation))
    x = word_tokenize(x)
    x = [word for word in x if not word in stopwords.words('english')]

    return (" ").join(x)

df_wordcloud['summary'] = df_wordcloud['summary'].apply(remove_non_utf8)
df_wordcloud.to_csv('data/topic_summary_joined.csv')

In [25]:
df_wordcloud = pd.read_csv('data/topic_summary_joined.csv', index_col ="Unnamed: 0")
df_wordcloud.head()

,topic,summary
0,TOPIC_EXCHANGE_RATE,The ICPXDR conversion rate set 359588 The ICPX...
1,TOPIC_GOVERNANCE,SetAuthorizedSubnetworks SetAuthorizedSubnetwo...
2,TOPIC_KYC,The proposal unlocks neurons associated listed...
3,TOPIC_NETWORK_CANISTER_MANAGEMENT,Upgrade ledger canister git commit 8a560f9510b...
4,TOPIC_NODE_ADMIN,Motivation Remove FR2 nodes network accordance...


In [26]:
def generate_wordcloud(df_wordcloud, topics):

    words = []
    for topic in topics:
        words.append(
            df_wordcloud.query(f"topic == '{topic}'")['summary'].values[0]
        )

    all_words = (' ').join(words)

    with open('data/all_words.txt', 'w') as f:
        f.write(all_words)

    wc = WordCloud(
        background_color='black',
        repeat=True,
        width=1024 * 2,
        height=768 * 2,
        max_words=200,
        colormap='Set2',
        font_path='fonts/impact.ttf'
    )
    wc.generate(all_words)

    return wc

In [27]:
def generate_word_freq(df_wordcloud, topics):

    words = []
    for topic in topics:
        words.append(
            df_wordcloud.query(f"{'topic'} == '{topic}'")['summary'].values[0]
        )
    all_words = [ele for ele in words if len(ele) > 20]
    all_words = (' ').join(all_words).upper()
    all_words = regex.sub(r'\b[0-9]+\b\W*', '', all_words)
    all_words = regex.sub(r'(?:^| )\w(?:$| )', ' ', all_words)

    def replace_digit(string):
        string = regex.sub(r'\d', '', string).strip()
        return string

    all_words = all_words.upper()

    tokens = nltk.word_tokenize(all_words)
    text = nltk.Text(tokens)
    freq = nltk.Counter(text).items()
    res = []
    for item in freq:
        res.append(item)

    res.sort(key=lambda x: x[1], reverse=True)
    return res

In [28]:
find_topics = df_wordcloud.topic.unique()
find_topics

array(['TOPIC_EXCHANGE_RATE', 'TOPIC_GOVERNANCE', 'TOPIC_KYC',
       'TOPIC_NETWORK_CANISTER_MANAGEMENT', 'TOPIC_NODE_ADMIN',
       'TOPIC_NODE_PROVIDER_REWARDS', 'TOPIC_SUBNET_MANAGEMENT'],
      dtype=object)

In [29]:
data = generate_word_freq(
    df_wordcloud=df_wordcloud,
    topics=find_topics
)

In [30]:
# create an empty pickle file
import os
filename = "figures/wordcloud.html"
os.makedirs(os.path.dirname(filename), exist_ok=True)

In [31]:
WordCloud(init_opts=opts.InitOpts()).add(
        series_name="Keywords",
        data_pair=data,
        word_size_range=[20, 80],
        textstyle_opts=opts.TextStyleOpts(
            font_family="impact",
            font_weight='bold',
            color='blue'
        ),
        rotate_step=90
    ).set_global_opts(
        # title_opts=opts.TitleOpts("Theme-dark"),
        title_opts=opts.TitleOpts(
            title="Keywords", title_textstyle_opts=opts.TextStyleOpts(font_size=23),
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    ).render('figures/wordcloud.html')

'/content/drive/MyDrive/ic-gov-visualization/figures/wordcloud.html'

In [32]:
import IPython
IPython.display.HTML(filename='figures/wordcloud.html')

## 3.2. Topics 

In [33]:
df = pd.read_pickle("proposals_no_empty.pkl")
topic_time_df = df[['proposal_id', 'topic',
                    'updated_at']].set_index('proposal_id')
topic_time_df.dtypes               

topic                 object
updated_at    datetime64[ns]
dtype: object

In [34]:
topic_time_df.columns

Index(['topic', 'updated_at'], dtype='object')

In [35]:
topic_time_df['updated_at']= pd.to_datetime(topic_time_df['updated_at']).dt.date
topic_time_df

,topic,updated_at
proposal_id,,
3,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05
4,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05
5,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05
6,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05
7,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05
...,...,...
82419,TOPIC_EXCHANGE_RATE,2022-09-24
82420,TOPIC_EXCHANGE_RATE,2022-09-24
82421,TOPIC_EXCHANGE_RATE,2022-09-24


In [36]:
topic_time_df['updated_at']= pd.to_datetime(topic_time_df['updated_at'])
topic_time_df

,topic,updated_at
proposal_id,,
3,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05
4,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05
5,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05
6,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05
7,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05
...,...,...
82419,TOPIC_EXCHANGE_RATE,2022-09-24
82420,TOPIC_EXCHANGE_RATE,2022-09-24
82421,TOPIC_EXCHANGE_RATE,2022-09-24


In [37]:
topic_time_df.dtypes

topic                 object
updated_at    datetime64[ns]
dtype: object

In [38]:
find_topics=list(topic_time_df.topic.unique())
find_topics

['TOPIC_NETWORK_CANISTER_MANAGEMENT',
 'TOPIC_GOVERNANCE',
 'TOPIC_SUBNET_MANAGEMENT',
 'TOPIC_EXCHANGE_RATE',
 'TOPIC_NODE_PROVIDER_REWARDS',
 'TOPIC_KYC',
 'TOPIC_NODE_ADMIN']

In [39]:
topic_time_df['updated_at'] = topic_time_df['updated_at']
topic_time_stats = topic_time_df.groupby(['updated_at','topic']).size().reset_index()
topic_time_stats.columns = ['update_at','topic','count']
topic_time_stats

,update_at,topic,count
0,2021-08-05,TOPIC_GOVERNANCE,8
1,2021-08-05,TOPIC_KYC,1
2,2021-08-05,TOPIC_NETWORK_CANISTER_MANAGEMENT,24
3,2022-01-27,TOPIC_EXCHANGE_RATE,499
4,2022-01-27,TOPIC_NODE_PROVIDER_REWARDS,9
5,2022-01-27,TOPIC_SUBNET_MANAGEMENT,9
6,2022-09-21,TOPIC_EXCHANGE_RATE,55
7,2022-09-22,TOPIC_EXCHANGE_RATE,108
8,2022-09-23,TOPIC_EXCHANGE_RATE,144
9,2022-09-24,TOPIC_EXCHANGE_RATE,96


In [40]:
topic_time_stats=topic_time_stats.pivot(index='update_at', columns='topic', values='count').fillna(0).reset_index()
topic_time_stats

topic,update_at,TOPIC_EXCHANGE_RATE,TOPIC_GOVERNANCE,TOPIC_KYC,TOPIC_NETWORK_CANISTER_MANAGEMENT,TOPIC_NODE_ADMIN,TOPIC_NODE_PROVIDER_REWARDS,TOPIC_SUBNET_MANAGEMENT
0,2021-08-05,0.0,8.0,1.0,24.0,0.0,0.0,0.0
1,2022-01-27,499.0,0.0,0.0,0.0,0.0,9.0,9.0
2,2022-09-21,55.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-09-22,108.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-09-23,144.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2022-09-24,96.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2022-09-25,0.0,0.0,0.0,0.0,0.0,0.0,23.0
7,2022-09-26,0.0,2.0,0.0,0.0,2.0,0.0,12.0
8,2022-09-27,0.0,0.0,0.0,0.0,0.0,0.0,23.0
9,2022-12-02,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [41]:
topic_time_stats.columns

Index(['update_at', 'TOPIC_EXCHANGE_RATE', 'TOPIC_GOVERNANCE', 'TOPIC_KYC',
       'TOPIC_NETWORK_CANISTER_MANAGEMENT', 'TOPIC_NODE_ADMIN',
       'TOPIC_NODE_PROVIDER_REWARDS', 'TOPIC_SUBNET_MANAGEMENT'],
      dtype='object', name='topic')

In [42]:
topic_time_stats=topic_time_stats.set_index('update_at')
topic_time_stats

topic,TOPIC_EXCHANGE_RATE,TOPIC_GOVERNANCE,TOPIC_KYC,TOPIC_NETWORK_CANISTER_MANAGEMENT,TOPIC_NODE_ADMIN,TOPIC_NODE_PROVIDER_REWARDS,TOPIC_SUBNET_MANAGEMENT
update_at,,,,,,,
2021-08-05,0.0,8.0,1.0,24.0,0.0,0.0,0.0
2022-01-27,499.0,0.0,0.0,0.0,0.0,9.0,9.0
2022-09-21,55.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-09-22,108.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-09-23,144.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-09-24,96.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-09-25,0.0,0.0,0.0,0.0,0.0,0.0,23.0
2022-09-26,0.0,2.0,0.0,0.0,2.0,0.0,12.0
2022-09-27,0.0,0.0,0.0,0.0,0.0,0.0,23.0


In [43]:
all_days = pd.date_range(topic_time_stats.index.min(), topic_time_stats.index.max(), freq='D')
topic_time_stats=topic_time_stats.reindex(all_days).fillna(0)
topic_time_stats

topic,TOPIC_EXCHANGE_RATE,TOPIC_GOVERNANCE,TOPIC_KYC,TOPIC_NETWORK_CANISTER_MANAGEMENT,TOPIC_NODE_ADMIN,TOPIC_NODE_PROVIDER_REWARDS,TOPIC_SUBNET_MANAGEMENT
2021-08-05,0.0,8.0,1.0,24.0,0.0,0.0,0.0
2021-08-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-08-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-08-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-08-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2022-11-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-11-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-11-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
topic_time_stats['update_at'] = topic_time_stats.index
topic_time_stats.reset_index()

topic,index,TOPIC_EXCHANGE_RATE,TOPIC_GOVERNANCE,TOPIC_KYC,TOPIC_NETWORK_CANISTER_MANAGEMENT,TOPIC_NODE_ADMIN,TOPIC_NODE_PROVIDER_REWARDS,TOPIC_SUBNET_MANAGEMENT,update_at
0,2021-08-05,0.0,8.0,1.0,24.0,0.0,0.0,0.0,2021-08-05
1,2021-08-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-08-06
2,2021-08-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-08-07
3,2021-08-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-08-08
4,2021-08-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-08-09
...,...,...,...,...,...,...,...,...,...
480,2022-11-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-11-28
481,2022-11-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-11-29
482,2022-11-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-11-30
483,2022-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-01


In [45]:
topic_time_stats=pd.melt(topic_time_stats, id_vars='update_at', value_vars=find_topics)

In [46]:
topic_time_stats

,update_at,topic,value
0,2021-08-05,TOPIC_NETWORK_CANISTER_MANAGEMENT,24.0
1,2021-08-06,TOPIC_NETWORK_CANISTER_MANAGEMENT,0.0
2,2021-08-07,TOPIC_NETWORK_CANISTER_MANAGEMENT,0.0
3,2021-08-08,TOPIC_NETWORK_CANISTER_MANAGEMENT,0.0
4,2021-08-09,TOPIC_NETWORK_CANISTER_MANAGEMENT,0.0
...,...,...,...
3390,2022-11-28,TOPIC_NODE_ADMIN,0.0
3391,2022-11-29,TOPIC_NODE_ADMIN,0.0
3392,2022-11-30,TOPIC_NODE_ADMIN,0.0
3393,2022-12-01,TOPIC_NODE_ADMIN,0.0


In [47]:
topic_time_stats.columns

Index(['update_at', 'topic', 'value'], dtype='object')

In [48]:
topic_time_stats['update_at']=pd.to_datetime(topic_time_stats['update_at']).dt.strftime('%Y-%m-%d')


In [49]:
topic_time_stats.columns 

Index(['update_at', 'topic', 'value'], dtype='object')

In [50]:
topic_time_stats.columns = ['order','name','value']

In [51]:
topic_time_stats['value'] = topic_time_stats['value'].astype('float')
topic_time_stats.dtypes

order     object
name      object
value    float64
dtype: object

In [52]:
topic_time_stats=topic_time_stats.sort_values(by=['order'])
topic_time_stats

,order,name,value
0,2021-08-05,TOPIC_NETWORK_CANISTER_MANAGEMENT,24.0
1455,2021-08-05,TOPIC_EXCHANGE_RATE,0.0
2425,2021-08-05,TOPIC_KYC,1.0
970,2021-08-05,TOPIC_SUBNET_MANAGEMENT,0.0
1940,2021-08-05,TOPIC_NODE_PROVIDER_REWARDS,0.0
...,...,...,...
969,2022-12-02,TOPIC_GOVERNANCE,0.0
1454,2022-12-02,TOPIC_SUBNET_MANAGEMENT,0.0
1939,2022-12-02,TOPIC_EXCHANGE_RATE,0.0
484,2022-12-02,TOPIC_NETWORK_CANISTER_MANAGEMENT,2.0


In [53]:
import os

if not os.path.exists("figures/area_fraction.png"):
    os.mkdir("figures/area_fraction.png")

In [58]:
import plotly.express as px
fig = px.area(topic_time_stats, x="order", y="value", color='name',groupnorm='fraction')
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.write_html("figures/area_fraction.html")
fig.write_image("figures/area_fraction.png")
fig.show()

In [55]:
import plotly.express as px
fig = px.area(topic_time_stats, x="order", y="value", color='name',groupnorm='percent')
# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)
fig.write_html("figures/area_percent.html")
fig.write_image("figures/area_percent.png")
fig.show()

In [59]:
import plotly.express as px
fig = px.area(topic_time_stats, x="order", y="value", color='name',groupnorm='')
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)
fig.write_html("figures/area_count.html")
fig.write_image("figures/area_count.png")
fig.show()

## 3.3. Network

In [62]:
proposer_network = df[['proposal_id', 'proposer',
                       'known_neurons_ballots', 'updated_at']]
proposer_network

,proposal_id,proposer,known_neurons_ballots,updated_at
0,3,35.0,[],2021-08-05 15:50:43.155180
1,4,35.0,[],2021-08-05 15:50:43.157946
2,5,35.0,[],2021-08-05 15:50:43.158920
3,6,35.0,[],2021-08-05 15:50:43.159834
4,7,35.0,[],2021-08-05 15:50:43.160766
...,...,...,...,...
1012,82419,26.0,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...",2022-09-24 16:03:27.550190
1013,82420,25.0,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...",2022-09-24 16:03:27.530476
1014,82421,24.0,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...",2022-09-24 16:03:27.510036
1015,82422,22.0,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...",2022-09-24 16:03:27.490062


In [69]:
proposer_network['known_neurons_ballots']=proposer_network['known_neurons_ballots'].astype('str')

In [70]:
proposer_network['known_neurons_ballots'] = proposer_network['known_neurons_ballots'].map(lambda x: x.lstrip('[').rstrip(']'))
proposer_network

,proposal_id,proposer,known_neurons_ballots,updated_at
0,3,35.0,,2021-08-05 15:50:43.155180
1,4,35.0,,2021-08-05 15:50:43.157946
2,5,35.0,,2021-08-05 15:50:43.158920
3,6,35.0,,2021-08-05 15:50:43.159834
4,7,35.0,,2021-08-05 15:50:43.160766
...,...,...,...,...
1012,82419,26.0,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2022-09-24 16:03:27.550190
1013,82420,25.0,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2022-09-24 16:03:27.530476
1014,82421,24.0,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2022-09-24 16:03:27.510036
1015,82422,22.0,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2022-09-24 16:03:27.490062


In [71]:
proposer_network = proposer_network[proposer_network['known_neurons_ballots'].apply(len)!=0]
proposer_network['proposer'] = proposer_network['proposer'].astype(int)
proposer_network

,proposal_id,proposer,known_neurons_ballots,updated_at
30,220,35,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2021-08-05 15:50:43.181794
31,275,46,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2021-08-05 15:50:43.182720
32,293,16,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2021-08-05 15:50:43.183702
33,302,46,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2021-08-05 15:50:43.184697
34,7877,23,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2022-01-27 19:34:24.376087
...,...,...,...,...
1012,82419,26,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2022-09-24 16:03:27.550190
1013,82420,25,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2022-09-24 16:03:27.530476
1014,82421,24,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2022-09-24 16:03:27.510036
1015,82422,22,"{'id': '27', 'name': 'DFINITY Foundation', 'vo...",2022-09-24 16:03:27.490062


In [73]:
import ast

In [74]:
# build proposer roster
voters = proposer_network['known_neurons_ballots'].values.flatten()
voters_temp = set()
voters_id = set()
for voter_group in tqdm(voters):
    voter_group = ast.literal_eval(voter_group)
    for voter in voter_group:
        voters_id.add(int(voter.get('id')))
        voter = (voter.get('id'), voter.get('name'))
        voters_temp.add(voter)

voters = voters_temp

proposer_set = set()
for proposal in proposer_network.itertuples():
    proposer_id = proposal.proposer
    proposer_set.add(int(proposer_id))

100%|██████████| 987/987 [00:00<00:00, 3483.56it/s]


In [75]:
voters

{('10843833286193887500', 'Anvil'),
 ('12860062727199510685', 'ysyms'),
 ('13538714184009896865', '8yeargangDAO'),
 ('14231996777861930328', 'ICDevs.org'),
 ('27', 'DFINITY Foundation'),
 ('28', 'Internet Computer Association'),
 ('428687636340283207', 'CryptoIsGood'),
 ('4966884161088437903', 'ICP Maximalist Network'),
 ('55674167450360693', 'ICPL.app'),
 ('5728549712200490799', 'ICPMANUAL'),
 ('5967494994762486275', 'Arthur’s Neuron (used to be cycle_dao)'),
 ('6362091663310642824', 'RawTech Venture'),
 ('7766735497505253681', "The Fools' Court"),
 ('8777656085298269769', 'Paul Young'),
 ('8959053254051540391', 'The Accumulators’ Neuron')}

In [76]:
proposer_set

{-9223372036854775808,
 16,
 19,
 22,
 23,
 24,
 25,
 26,
 31,
 35,
 39,
 40,
 46,
 47,
 48,
 50,
 59,
 69}

In [77]:
network_df = pd.DataFrame(0, columns=list(voters_id), index=list(proposer_set))
network_df

,12860062727199510685,5967494994762486275,13538714184009896865,428687636340283207,8959053254051540391,6362091663310642824,8777656085298269769,10843833286193887500,4966884161088437903,5728549712200490799,7766735497505253681,55674167450360693,27,28,14231996777861930328
35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
69,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [78]:
import ast

In [79]:
for proposal in tqdm(proposer_network.itertuples(), total=len(proposer_network)):
    voters = proposal.known_neurons_ballots
    proposer_id = int(proposal.proposer)
    voters = ast.literal_eval(voters)
    for voter in voters:
        voter_id = int(voter.get('id'))
        vote = voter.get('vote')
        network_df.loc[proposer_id, voter_id] += 1

100%|██████████| 987/987 [00:01<00:00, 667.73it/s]


In [80]:
network_df

,12860062727199510685,5967494994762486275,13538714184009896865,428687636340283207,8959053254051540391,6362091663310642824,8777656085298269769,10843833286193887500,4966884161088437903,5728549712200490799,7766735497505253681,55674167450360693,27,28,14231996777861930328
35,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0
69,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
39,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
40,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44
46,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0
47,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
16,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
48,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0
50,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0
19,0,0,0,0,0,0,0,0,0,0,0,0,9,9,0


### 3.3.1. Heatmap

In [82]:
network_df = network_df.astype(float)
row_names = ('proposer'+' '+ network_df.index.astype('str')).to_list()
col_names = ('voter'+' '+ network_df.columns.astype('str')).to_list()
network_matrix=network_df.values.tolist()


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0],
 [44.0,
  44.0,
  44.0,
  44.0,
  44.0,
  44.0,
  44.0,
  44.0,
  44.0,
  44.0,
  44.0,
  44.0,
  44.0,
  44.0,
  44.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0],
 [9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 9.0, 0.0],
 [69.0,
  69.0,
  69.0,
  69.0,
  69.0,
  69.0,
  69.0,
  69.0,
  69.0,
  69.0,
  69.0,
  69.0,
  166.0,
  166.0,
  69.0],
 [88.0,
  88.0,
  88.0,
  88.0,
  88.0,
  88.0,
  88.0

In [93]:
import plotapi #https://plotapi.com/gallery/posts/showcase/pokemon-types-with-plotapi-heatmap/
plotapi.api_key("9bba9d28-5956-49c9-9e77-70ef4084b3ba")
from plotapi import HeatMap
HeatMap(network_matrix,row_names=row_names, col_names=col_names).to_html("figures/heatmap.html")
HeatMap(network_matrix,row_names=row_names, col_names=col_names).to_png("figures/heatmap.png")
HeatMap(network_matrix,row_names=row_names, col_names=col_names).to_mp4("figures/heatmap.mp4")
HeatMap(network_matrix,row_names=row_names, col_names=col_names).to_png("figures/heatmap.png")
HeatMap(network_matrix,row_names=row_names, col_names=col_names).to_pdf("figures/heatmap.pdf")
HeatMap(network_matrix,row_names=row_names, col_names=col_names).show()

Your PlotAPI API key has been saved in your local environment. You will not need to set it again.


### 3.3.2 Chord Chart

In [97]:
links = []
nodes = []

for item in network_df.itertuples():
    proposer = f'Proposer {item.Index}'
    nodes.append(dict(
        name=proposer,
        group='left'
    ))
    for voter in network_df.columns:
        amount = int(network_df.loc[item.Index, voter])
        voter = f'Voter {voter}'
        links.append(dict(
                source=voter,
                target=proposer,
                value=amount
        ))

for voter in network_df.columns:
    voter = f'Voter {voter}'
    nodes.append(dict(
        name=voter,
        group='right'
    ))


In [131]:
# https://plotapi.com/gallery/posts/chord/bipartite-divided-labels/
import plotapi
plotapi.api_key("9bba9d28-5956-49c9-9e77-70ef4084b3ba")
from plotapi import SplitChord
SplitChord(links,nodes).show()

Your PlotAPI API key has been saved in your local environment. You will not need to set it again.


In [135]:
import plotapi
plotapi.api_key("9bba9d28-5956-49c9-9e77-70ef4084b3ba")
from plotapi import SplitChord
SplitChord(links,nodes).to_html('figures/chord.html')
SplitChord(links,nodes).to_png('figures/chord.png')
SplitChord(links,nodes).to_mp4('figures/chord.mp4')
SplitChord(links,nodes).to_pdf('figures/chord.pdf')

Your PlotAPI API key has been saved in your local environment. You will not need to set it again.


In [136]:
import IPython
IPython.display.HTML(filename='figures/chord.html')

### 3.3.3 Sankey

In [137]:
links

[{'source': 'Voter 12860062727199510685', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 5967494994762486275', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 13538714184009896865', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 428687636340283207', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 8959053254051540391', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 6362091663310642824', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 8777656085298269769', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 10843833286193887500', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 4966884161088437903', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 5728549712200490799', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 7766735497505253681', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 55674167450360693', 'target': 'Proposer 35', 'value': 0},
 {'source': 'Voter 27', 'target': 'Proposer 35', 'va

In [138]:
# https://datacrayon.com/posts/plotapi/sankey/customising-the-nodes/
import plotapi
plotapi.api_key("9bba9d28-5956-49c9-9e77-70ef4084b3ba")
from plotapi import Sankey
Sankey(links, node_font_size=8).show()

Your PlotAPI API key has been saved in your local environment. You will not need to set it again.


In [140]:
import plotapi
plotapi.api_key("9bba9d28-5956-49c9-9e77-70ef4084b3ba")
from plotapi import Sankey
Sankey(links, node_font_size=8).to_html("figures/Sankey.html")
Sankey(links, node_font_size=8).to_png("figures/Sankey.png")
Sankey(links, node_font_size=8).to_mp4("figures/Sankey.mp4")
Sankey(links, node_font_size=8).to_pdf("figures/Sankey.pdf")

Your PlotAPI API key has been saved in your local environment. You will not need to set it again.


### 3.3.4 Terminus

In [146]:
few_links = list(filter(lambda x: x['value'] > 10, links))

In [154]:
import plotapi
plotapi.api_key("9bba9d28-5956-49c9-9e77-70ef4084b3ba")
from plotapi import Terminus
Terminus(few_links, show_stats=False).show()

Your PlotAPI API key has been saved in your local environment. You will not need to set it again.


In [164]:
import plotapi
plotapi.api_key("9bba9d28-5956-49c9-9e77-70ef4084b3ba")
from plotapi import Terminus
Terminus(few_links, show_stats=False).to_html("figures/Terminus.html")

Your PlotAPI API key has been saved in your local environment. You will not need to set it again.


In [166]:
import IPython
IPython.display.HTML(filename='figures/Terminus.html')

## 3.4: Proposal Summary Network

In [167]:
df = pd.read_pickle("proposals_no_empty.pkl")
df

,action,action_nns_function,deadline_timestamp_seconds,decided_timestamp_seconds,executed_timestamp_seconds,failed_timestamp_seconds,id,known_neurons_ballots,latest_tally,payload,...,proposer,reject_cost_e8s,reward_status,settled_at,status,summary,title,topic,updated_at,url
0,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340878,1620340878,0.0,1,[],"{'no': 0, 'timestamp_seconds': 1620340878, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade ledger canister to git commit 8a560f95...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.155180,https://github.com/dfinity/nns-proposals/blob/...
1,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340965,1620340965,0.0,2,[],"{'no': 0, 'timestamp_seconds': 1620340965, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade registry canister to git commit 8a560f...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.157946,https://github.com/dfinity/nns-proposals/blob/...
2,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341025,1620341025,0.0,3,[],"{'no': 0, 'timestamp_seconds': 1620341025, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade governance canister to git commit 8a56...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.158920,https://github.com/dfinity/nns-proposals/blob/...
3,ExecuteNnsFunction,NnsRootUpgrade,NaN,1620341096,1620341110,0.0,4,[],"{'no': 0, 'timestamp_seconds': 1620341096, 'to...",{},...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade root canister to git commit 8a560f9510...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.159834,https://github.com/dfinity/nns-proposals/blob/...
4,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341144,1620341144,0.0,5,[],"{'no': 0, 'timestamp_seconds': 1620341144, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade cycles-minting canister to git commit ...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.160766,https://github.com/dfinity/nns-proposals/blob/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,ExecuteNnsFunction,IcpXdrConversionRate,1.664003e+09,1663960248,1663960248,0.0,68169,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663960248, 'to...","{'data_source': {'icp': ['Binance'], 'sdr': 'x...",...,26.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7801.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.550190,
1013,ExecuteNnsFunction,IcpXdrConversionRate,1.664004e+09,1663960883,1663960883,0.0,68170,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663960883, 'to...","{'data_source': {'icp': ['Huobi'], 'sdr': 'xe....",...,25.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7723.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.530476,
1014,ExecuteNnsFunction,IcpXdrConversionRate,1.664005e+09,1663961552,1663961552,0.0,68171,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663961552, 'to...","{'data_source': {'icp': ['Binance'], 'sdr': 'x...",...,24.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7646.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.510036,
1015,ExecuteNnsFunction,IcpXdrConversionRate,1.664005e+09,1663962087,1663962087,0.0,68172,"[{'id': '27', 'name': 'DFINITY Foundation', 'v...","{'no': 0, 'timestamp_seconds': 1663962087, 'to...","{'data_source': {'icp': ['Huobi'], 'sdr': 'xe....",...,22.0,1000000000,SETTLED,2022-09-24 16:00:00,EXECUTED,,The ICP/XDR conversion rate is set to 4.7646.,TOPIC_EXCHANGE_RATE,2022-09-24 16:03:27.490062,


In [168]:
df_network = df[df['topic'] != 'TOPIC_EXCHANGE_RATE']['summary']

In [169]:
network_all_words = ('. '.join(list(df_network.values)))
network_all_words = regex.sub(r'\b[0-9]+\b\W*', '', network_all_words)
network_all_words = regex.sub(r'(?:^| )\w(?:$| )', ' ', network_all_words)

In [170]:
def remove_url(txt):
    url_pattern = regex.compile(r'https?://\S+|www\.\S+')
    no_url = url_pattern.sub(r'', txt)

    return no_url


def stem_tokens(tokens):
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens


def lemmatize_tokens(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens


In [171]:
text_no_urls = remove_url(network_all_words)
text_no_urls = text_no_urls.encode('utf-8', errors='ignore').decode('utf-8')
words_in_text = text_no_urls.split()
stop_words = set(stopwords.words('english'))
words_nsw = [word for word in words_in_text if not word in stop_words]
words_nsw = [ele for ele in words_nsw if len(ele) < 20]


In [172]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [173]:
lemmatizer = WordNetLemmatizer()
words_nsw_nc = [lemmatizer.lemmatize(w) for w in words_nsw]
words_nsw_nc = [
    word for word in words_nsw_nc if word not in string.punctuation]

In [174]:
bigram = list(nltk.bigrams(words_nsw_nc))
bigram_counts = collections.Counter(bigram).most_common(100)

In [175]:
G = nx.Graph()
for edge in bigram_counts:
    link = edge[0]
    count = edge[1]
    
    G.add_edge(link[0], link[1], weight=count)

In [179]:
fig = ig.plot(
    G,
    node_opacity=0.8,
)
fig.update_layout(
    dict(
        width=800,
        height=600,
    )
)

with open('figures/network.html', 'w') as f:
    f.write(fig.to_html())

fig.show()
fig.write_html("figures/bigram.html")
fig.write_image("figures/bigram.png")